In [1]:
import numpy as np
from pyscf import gto, scf, ao2mo

Be $A$ a matrix and $A = U^T \lambda U$, with $\lambda_{ij} = \lambda_i \delta_{ij}$ where $\lambda$ are the eigenvalues of $A$, then
$A^{\frac{1}{2}}= U^T \lambda^{\frac{1}{2}} U$ in the sense, that $$A^{\frac{1}{2}} A^{\frac{1}{2}} = A$$ since $$A^{\frac{1}{2}} A^{\frac{1}{2}} = U^T \lambda^{\frac{1}{2}} U U^T \lambda^{\frac{1}{2}} U = U^T \lambda U$$

In [2]:
def sqrt_matrix(in_mat):
    eigval, eigvec = np.linalg.eigh(in_mat)
    sqrt_eigval = np.sqrt(eigval)
    return(np.matmul(eigvec.T,np.matmul(np.diag(sqrt_eigval),eigvec)))


$$
\begin{split}
E_x & = \sum_i^N \sum_j^N \sum_{\mu}^M \sum_{\nu}^M \sum_{\kappa}^M \sum_{\lambda}^M c_{i,\mu} c_{j,\nu} c_{i,\kappa} c_{j,\lambda} \left [ \mu \nu | \kappa \lambda \right ] \\
E_x & = \sum_i^N \sum_j^N  \left [ i j | i j \right ] \\
E_x & = \sum_i^N \sum_j^N  \langle  i j | j i  \rangle 
\end{split}
$$

In [3]:
def exchange_energy(Fouridx, C, mol):
    energy = 0
    M = Fouridx.shape[0]
    N = mol.nelec[0]
    for i in  range(0,N):
        for j in range(0,N):
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += C[mu,i]*C[nu,j]*C[kappa,i]*C[lamda,j]*Fouridx[mu,nu,kappa,lamda]

    return energy


$$
\begin{split}
E_H & = \sum_i^N \sum_j^N \sum_{\mu}^M \sum_{\nu}^M \sum_{\kappa}^M \sum_{\lambda}^M c_{i,\mu} c_{j,\nu} c_{i,\kappa} c_{j,\lambda} \left [ \mu \nu | \kappa \lambda \right ] \\
E_H & = \sum_i^N \sum_j^N  \left [ i i | j j \right ] \\
E_H & = \sum_i^N \sum_j^N  \langle  i j | i j \rangle 
\end{split}
$$

In [4]:
def hartree_energy(Fouridx, C, mol):
    energy = 0
    M = Fouridx.shape[0]
    N = mol.nelec[0]
    for i in  range(0,N):
        for j in range(0,N):
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += C[mu,i]*C[nu,i]*C[kappa,j]*C[lamda,j]*Fouridx[mu,nu,kappa,lamda]

    return energy


In contrast to the previous expression for the hartree energy, this expression runs over all orbitals {$\phi_a$}, which are the natural orbitals.  $n_a$ are the occupation numbers. $c_{a, \mu}$ are the coefficients of the matrix that diagonalizes $P$, which is the basis set representation of $\gamma^{(1)}$.

$$
\begin{split}
E_H[\gamma^{(1)}] & = \sum_a^M \sum_b^M n_a n_b \sum_{\mu}^M \sum_{\nu}^M \sum_{\kappa}^M \sum_{\lambda}^M  c_{a,\mu} c_{b,\nu} c_{a,\kappa} c_{b,\lambda} \left [ \mu \nu | \kappa \lambda \right ] \\
E_H[\gamma^{(1)}] & = \iint \sum_a^M \sum_b^M n_a n_b \frac{\phi_a(r)^{\ast} \phi_a(r) \phi_b(r')^{\ast}  \phi_b(r')}{|r-r'|} dr dr' \\
E_H[\gamma^{(1)}] & = \iint \frac{\gamma^{(1)}(r,r) \gamma^{(1)}(r',r')}{|r-r'|} dr dr' \\
E_H[\gamma^{(1)}] & = \sum_a^M \sum_b^M n_a n_b \left [ a a  | b b \right ] \\
E_H[\gamma^{(1)}] & = \sum_a^N \sum_b^N n_a n_b \langle  a b | a b \rangle 
\end{split}
$$

In [5]:
def ONERDMFT_hartree_energy(Fouridx, C, n, mol):
    energy = 0
    M = Fouridx.shape[0]
    N = mol.nelec[0]
    for i in  range(0,M):
        for j in range(0,M):
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += n[i]*n[j]*C[mu,i]*C[nu,i]*C[kappa,j]*C[lamda,j]*Fouridx[mu,nu,kappa,lamda]

    return energy


In [6]:
def ONERDMFT_exchange_energy(Fouridx, C, n, mol):
    energy = 0
    M = Fouridx.shape[0]
    N = mol.nelec[0]
    for i in  range(0,M):
        for j in range(0,M):
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += n[i]*n[j]*C[mu,i]*C[nu,j]*C[kappa,j]*C[lamda,i]*Fouridx[mu,nu,kappa,lamda]

    return energy

In [7]:
def ONERDMFT_Mueller_functional(Fouridx, C, n, mol):
    energy = 0
    M = Fouridx.shape[0]
    N = mol.nelec[0]
    for i in  range(0,M):
        for j in range(0,M):
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += np.sqrt(n[i]*n[j])*C[mu,i]*C[nu,j]*C[kappa,j]*C[lamda,i]*Fouridx[mu,nu,kappa,lamda]

    return energy

In [8]:
mol = gto.Mole()
mol.atom = """
    Ne    0.    0.    0.
"""
# this basis has 2 functions for Helium
mol.basis = "6-31g" #mol.basis = "ccpvdz", mol.basis = "sto-6g"
mol.build()

# the 2 electron integrals \langle \mu \nu | \kappa \lambda \rangle have M^4 in the case of  case 16 distinct elements
eri = mol.intor('int2e')
print(f"The System {mol.atom} has {eri.size} or {eri.shape[0]}^4 elements in 2-electron-intergrals/4-index-integrals matrix with the {mol.basis}-basis")



The System 
    Ne    0.    0.    0.
 has 6561 or 9^4 elements in 2-electron-intergrals/4-index-integrals matrix with the 6-31g-basis


In [9]:
print("Overlap Integrals")
S = mol.intor('int1e_ovlp')
for i in range(0,S.shape[0]):
    for j in range(0,S.shape[1]):
        print(f"S_{i}{j} = {S[i,j]}" )
SMH = sqrt_matrix(S)

Overlap Integrals
S_00 = 0.9999999999999998
S_01 = 0.24901801230727033
S_02 = 0.17191310932176385
S_03 = 0.0
S_04 = 0.0
S_05 = 0.0
S_06 = 0.0
S_07 = 0.0
S_08 = 0.0
S_10 = 0.24901801230727028
S_11 = 0.9999999999999998
S_12 = 0.7590309056851063
S_13 = 0.0
S_14 = 0.0
S_15 = 0.0
S_16 = 0.0
S_17 = 0.0
S_18 = 0.0
S_20 = 0.17191310932176382
S_21 = 0.7590309056851063
S_22 = 0.9999999999999999
S_23 = 0.0
S_24 = 0.0
S_25 = 0.0
S_26 = 0.0
S_27 = 0.0
S_28 = 0.0
S_30 = 0.0
S_31 = 0.0
S_32 = 0.0
S_33 = 1.0000000000000002
S_34 = 0.0
S_35 = 0.0
S_36 = 0.4929819049679909
S_37 = 0.0
S_38 = 0.0
S_40 = 0.0
S_41 = 0.0
S_42 = 0.0
S_43 = 0.0
S_44 = 1.0000000000000002
S_45 = 0.0
S_46 = 0.0
S_47 = 0.4929819049679909
S_48 = 0.0
S_50 = 0.0
S_51 = 0.0
S_52 = 0.0
S_53 = 0.0
S_54 = 0.0
S_55 = 1.0000000000000002
S_56 = 0.0
S_57 = 0.0
S_58 = 0.4929819049679909
S_60 = 0.0
S_61 = 0.0
S_62 = 0.0
S_63 = 0.4929819049679909
S_64 = 0.0
S_65 = 0.0
S_66 = 1.0000000000000004
S_67 = 0.0
S_68 = 0.0
S_70 = 0.0
S_71 = 0.0
S_72 = 0

In [10]:
## Run Hartree-Fock.
mf = scf.RHF(mol)
mf.kernel()

print("*"*24)
print("MO-Coefficent matrix")
for mu,AO in enumerate(mf.mo_coeff):
    print(f"Coefficients of mu={mu} {AO}")
print("*"*24)

#print("den ?")
#print(np.matmul(mf.mo_coeff.T,mf.mo_coeff))
#print("*"*24)

converged SCF energy = -128.473876870668
************************
MO-Coefficent matrix
Coefficients of mu=0 [ 0.99550796 -0.24482371  0.         -0.         -0.          0.
  0.         -0.         -0.12617152]
Coefficients of mu=1 [ 0.02008192  0.54555307  0.         -0.         -0.          0.
  0.         -0.          1.46444498]
Coefficients of mu=2 [-0.00378329  0.5485072   0.         -0.         -0.          0.
  0.         -0.         -1.43529233]
Coefficients of mu=3 [-0.        -0.         0.         0.6902442 -0.        -0.9190328
 -0.        -0.        -0.       ]
Coefficients of mu=4 [-0.00000000e+00  0.00000000e+00  6.76783419e-01 -6.73442924e-18
  1.35651260e-01 -1.54478002e-17  3.11075242e-01 -8.64785218e-01
 -0.00000000e+00]
Coefficients of mu=5 [-0.00000000e+00 -0.00000000e+00 -1.35651260e-01 -1.75798261e-18
  6.76783419e-01 -4.03255616e-18 -8.64785218e-01 -3.11075242e-01
 -0.00000000e+00]
Coefficients of mu=6 [-0.          0.          0.          0.45931699 -0.       

Energy Components from PySCF Tools:
$$ T + V_{eK} = Tr[h \gamma^{(1)}];  U = \frac{1}{2} Tr[J \gamma^{(1)}]; E_x = -\frac{1}{4} Tr[K \gamma^{(1)}]$$


In [11]:
# get j, k and gamma (1RDM) matrix from hf, 
J = mf.get_j()
K = mf.get_k()
h = mf.get_hcore()
C = mf.mo_coeff
gamma = mf.make_rdm1()

# calculate the energy components to see what they are from the matrices the mf object offers you 
print("Energy Components from PySCF Tools:")
print(f"h_0 = {np.trace(np.matmul(h,gamma))}; U = {1/2*np.trace(np.matmul(J,gamma))}; E_x =  {1/4.*np.trace(np.matmul(K,gamma))}")
print(f"h_0 + U + E_x = {np.trace(np.matmul(h,gamma))+1/2.*np.trace(np.matmul(J,gamma))-1/4.*np.trace(np.matmul(K,gamma))}")

Energy Components from PySCF Tools:
h_0 = -182.62284394764518; U = 66.27654288017766; E_x =  12.127575803200871
h_0 + U + E_x = -128.4738768706684


Energy Components 4IDX:

$$U = 2 E_h[C]; E_x = -E_x[C]$$

In [12]:
# this should also work
print("Energy Components from direct calculations:")
print(f"U = {2*hartree_energy(eri, mf.mo_coeff, mol)} E_x = {exchange_energy(eri, mf.mo_coeff, mol)} ")

Energy Components from direct calculations:
U = 66.27654288017764 E_x = 12.127575803200898 


In [13]:
print("Text-Book Gamma")
N = mol.nelec[0]
print(N)
MPgamma=np.matmul(C[:,0:N],C[:,0:N].T)*2
for mu,AO in enumerate(MPgamma):
    print(f"Coefficients of mu={mu} {AO}")

print("PySCF Gamma")
for mu,AO in enumerate(gamma):
    print(f"Coefficients of mu={mu} {AO}")




Text-Book Gamma
5
Coefficients of mu=0 [ 2.1019495  -0.22714523 -0.27610773  0.          0.          0.
  0.          0.          0.        ]
Coefficients of mu=1 [-0.22714523  0.59606287  0.59832762  0.          0.          0.
  0.          0.          0.        ]
Coefficients of mu=2 [-0.27610773  0.59832762  0.60174891  0.          0.          0.
  0.          0.          0.        ]
Coefficients of mu=3 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  9.52874120e-01
 -9.29680149e-18 -2.42687461e-18  6.34081779e-01  1.88583017e-17
  6.22751810e-19]
Coefficients of mu=4 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -9.29680149e-18
  9.52874120e-01 -2.08780887e-16 -6.18647553e-18  6.34081779e-01
 -2.35671212e-16]
Coefficients of mu=5 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -2.42687461e-18
 -2.08780887e-16  9.52874120e-01 -1.61494256e-18 -3.92891796e-17
  6.34081779e-01]
Coefficients of mu=6 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  6.34081779e-01
 -6.18647553e-18 

In [14]:
# this serves to show  that, not C is orthogonal, but S^{-1/2} C
print(np.matmul(S,np.matmul(C,C.T)))

[[ 1.00000000e+00 -9.41500838e-17  5.30456318e-17  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 8.15279202e-17  1.00000000e+00 -5.37944629e-16  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 1.75632936e-16 -4.65989154e-16  1.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00
   2.95914370e-33  8.73731708e-34 -2.52680954e-16 -7.02834211e-33
  -4.26544526e-35]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.44002592e-33
   1.00000000e+00 -8.39933392e-17 -3.94685420e-33 -1.43216982e-16
  -7.66761573e-17]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  2.17729249e-18
   2.03807179e-17  1.00000000e+00 -4.41657689e-18 -2.88396012e-16
  -2.51122624e-16]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -6.71719819e-17
  -1.46618335e-17 -3.8273842

In [15]:
occu, naturalC = np.linalg.eigh(gamma)
print(f"should be natural occupation numbers, the sum of occupation numbers, i.e. N is {np.sum(occu)}")
print(occu)
print("*"*24)
print("Natural Orbital LC-Coefficent matrix")
for mu,AO in enumerate(naturalC):
    print(f"Coefficients of mu={mu} {AO}")
print("*"*24)
print(np.matmul(naturalC, naturalC.T))

should be natural occupation numbers, the sum of occupation numbers, i.e. N is 7.424216223676513
[-7.57831784e-17  0.00000000e+00  2.91944005e-17  4.64361237e-16
  1.07410947e+00  1.37481831e+00  1.37481831e+00  1.37481831e+00
  2.22565181e+00]
************************
Natural Orbital LC-Coefficent matrix
Coefficients of mu=0 [-0.          0.          0.01691819  0.          0.3269097   0.
 -0.         -0.         -0.94490413]
Coefficients of mu=1 [-0.          0.         -0.70512623  0.          0.67391634  0.
 -0.         -0.          0.22053066]
Coefficients of mu=2 [-0.          0.          0.70887995  0.          0.66254571  0.
 -0.         -0.          0.24191405]
Coefficients of mu=3 [-0.          0.55399373  0.          0.          0.          0.
 -0.83252084 -0.         -0.        ]
Coefficients of mu=4 [ 0.31631045  0.          0.          0.45481508  0.         -0.43975563
  0.          0.70689881 -0.        ]
Coefficients of mu=5 [-4.54815083e-01  3.18637713e-18  0.00000000

In [16]:
# this is always off by a factor of 4 but it works, i.e. I get reasonable numbers
#which means here I deal as before only with quantities that are given 
# with respect to a basis set, never the quanities itselff.

print("Hartree DM/4I", 1/2*ONERDMFT_hartree_energy(eri, naturalC, occu, mol), 2*hartree_energy(eri, mf.mo_coeff, mol), ONERDMFT_hartree_energy(eri, naturalC, occu, mol)/hartree_energy(eri, mf.mo_coeff, mol))
print("Exchange DM/4I", 1/4*ONERDMFT_exchange_energy(eri, naturalC, occu, mol), exchange_energy(eri, mf.mo_coeff, mol), ONERDMFT_exchange_energy(eri, naturalC, occu, mol)/exchange_energy(eri, mf.mo_coeff, mol))

Hartree DM/4I 66.27654288017764 66.27654288017764 4.0
Exchange DM/4I 12.127575803200886 12.127575803200898 3.999999999999996


In [17]:
J_PY = 1/2*np.trace(np.matmul(J,gamma))
J_4I = 2*hartree_energy(eri, mf.mo_coeff, mol)
J_DM = 1/2*ONERDMFT_hartree_energy(eri, naturalC, occu, mol)

print(J_PY, J_4I, J_DM)

66.27654288017766 66.27654288017764 66.27654288017764


In [19]:
for i, n  in enumerate(occu):
    if abs(n) < 1e-10:
        occu[i] = 0 

print("Mueller Correlation Energy:", 1/4*ONERDMFT_Mueller_functional(eri, naturalC, occu, mol)-1/4*ONERDMFT_exchange_energy(eri, naturalC, occu, mol))

Mueller Correlation Energy: -4.473733960556157
